In [ ]:
import pandas as pd
import ixmp4
import nomenclature
import pyam

In [ ]:
iamc_args = dict(
    model="State of CDR (2024) - Demonstration and Upscaling",
    scenario="Historical Reference",
)

In [ ]:
dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
data_start_up = pd.read_excel("source/3_Demonstration/chapter-3-start-up-template.xlsx")

In [ ]:
data_start_up["Unit"].fillna("", inplace=True)
data_start_up.drop(columns="Description", inplace=True)

In [ ]:
data_start_up_1 = data_start_up[
    [("Start-Up Investment|Deals" == v or "Start-Up Investment|Funding" == v or "Share" in v) for v in data_start_up.Variable]
]

In [ ]:
data_start_up_2 = data_start_up[
    [("Start-Up Investment|Deals" != v and "Start-Up Investment|Funding" != v and "Share" not in v) for v in data_start_up.Variable]
].fillna(0)

In [ ]:
df = pyam.concat(
    [data_start_up_1, data_start_up_2],
    **iamc_args,
)

In [ ]:
df.rename(
    variable=dict(
        {
          (
              i, 
              (
                  i.replace("Start-Up Investment", "Start-Up")
                  .replace("BECCS", "Bioenergy with CCS")
                  .replace("DAC(CS)", "Direct Air Capture with CCS")
                  .replace("DOC(CS)", "Direct Ocean Capture")
                  .replace("Soil Carbon", "Soil Carbon Sequestration")
              )
          )
            for i in df.variable
        },
    ),
    inplace=True,
)

In [ ]:
df.filter(variable="Start-Up|Funding*").timeseries()

In [ ]:
startup_number = pd.read_csv("source/3_Demonstration/fig2a_v2.csv")

In [ ]:
df_startup_number = pyam.IamDataFrame(
    startup_number.rename(columns={"Year.founded": "Year", "Method": "Variable", "Num.startups": "value"}),
    **iamc_args,
    region="World",
    unit="",
)

In [ ]:
df_startup_number.rename(
    variable=dict(
        {
          (
              i, "Start-Up|Number of Firms|Removal|" +
              (
                  i
                  .replace("BECCS", "Bioenergy with CCS")
                  .replace("Blue carbon", "Blue Carbon")
                  .replace("DACCS", "Direct Air Capture with CCS")
                  .replace("DOCCS", "Direct Ocean Capture")
                  .replace("Soil carbon", "Soil Carbon Sequestration")
                  .replace("Biomass burial", "Biomass Burial")
                  .replace("Enhanced weathering", "Enhanced Weathering")
                  .replace("Marine biomass", "Biomass Sinking")
                  .replace("OAE", "Ocean Alkalinity Enhancement")
                  
              )
          )
            for i in df_startup_number.variable
        },
    ),
    inplace=True,
)

In [ ]:
df_startup_number.variable

In [ ]:
df_startup_number.plot.bar(stacked=True)

In [ ]:
df_startup_number.aggregate("Start-Up|Number of Firms|Removal", append=True)

In [ ]:
planned_projects = pd.read_csv("source/3_Demonstration/Chapter3_Fig4_Data_v1.csv")

In [ ]:
columns = ["DACCS", "BECCS", "Cement prod. with CCS ", "WtE with CCS", "Total BECCS", "Total"]

df_planned_projects = pyam.IamDataFrame(
    planned_projects[columns + ["Year"]],
    **iamc_args,
    value=columns,
    region="World",
    unit="t CO2",
).convert_unit("t CO2", "Mt CO2")

In [ ]:
df_planned_projects.rename(
    variable=dict(
        {
          (
              i, "Planned Projects|Removal" +
              (
                  i
                  .replace("BECCS", "|Bioenergy with CCS")
                  .replace("DACCS", "|Direct Air Capture with CCS")
                  .replace("Total", "")
                  
              )
          )
            for i in df_planned_projects.variable
        },
    ),
    inplace=True,
)

In [ ]:
announced_projects = pd.read_csv("source/3_Demonstration/Chapter3_Fig5_Data_v1.csv")

In [ ]:
announced_projects

In [ ]:
final_df = pyam.concat(
    [df, df_startup_number]
)

In [ ]:
dsd.validate(final_df)

In [ ]:
platform = ixmp4.Platform("socdr-dev")

In [ ]:
df.set_meta("Chapter 3", "Chapter")

In [ ]:
df

In [ ]:
#df.to_ixmp4(platform)

In [ ]:
run = platform.runs.get(**iamc_args)

In [ ]:
run.iamc.remove(run.iamc.tabulate())

In [ ]:
run.iamc.add(final_df.data)

In [ ]:
platform.units.create("-")

In [ ]:
df.rename(unit={"": "-"}, inplace=True)

In [ ]:
platform.units.tabulate()

In [ ]:
final_df.variable

In [ ]:
final_df.filter(variable="Start-Up|Number*").variable

In [ ]:
final_df.filter(variable="Start-Up|Deal*").variable